Factorization Machine

参考：
- [推荐系统遇上深度学习(一)--FM模型理论和实践](https://blog.csdn.net/jiangjiang_jian/article/details/80630873)
- [分解机(Factorization Machines)推荐算法原理](https://www.cnblogs.com/pinard/p/6370127.html)


推荐系统对商品的点击率进行预估建模时，特征处理环节，除了单特征的处理外，通常要对特征进行组合，对于特征组合来说，常用的两种方法为FM系列和Tree系列。 



一般的线性模型为
$$y= w_0 + \sum_{i=1}^n w_i x_i$$

从表达式中也可以看出，一般的线性模型没有考虑任何特征之间的联系。

在考虑特征两个特征的组合时，可以用下面的二项式形式表示

$$y= w_0 + \sum_{i=1}^n w_i x_i + \sum_{i=1}^{n-1} \ sum_{j = i+1}^n w_{ij}x_i x_j$$

FM模型比一般的多项式模型就多了二项式的部分。

但是，在模型求解释，很多特征的组合数据量非常小，甚至是没有的，特别是特征对特征进行了one hot方法的处理之后，这样组合特征的参数就没有办法求参，因此在求参时，也使用了类似因变量分解的方法：


![FM1](pic/FM1.png)
![FM2](pic/FM2.png)
![FM3](pic/FM3.png)



# 将列表转换为稀疏矩阵
从列表创建稀疏矩阵
参数：
- dic: 特征列表地点，key值为特征名
- ix: 
- p: 特征维度

In [1]:
from itertools import count
from collections import defaultdict
from scipy.sparse import csr

def vectorize_dic(dic,ix=None,p=None,n=0,g=0):
    """
    dic -- dictionary of feature lists. Keys are the name of features
    ix -- index generator (default None)
    p -- dimension of featrure space (number of columns in the sparse matrix) (default None)
    """
    if ix==None:
        ix = dict()
 
    nz = n * g
 
    col_ix = np.empty(nz,dtype = int)
 
    i = 0
    for k,lis in dic.items():
        for t in range(len(lis)):
            ix[str(lis[t]) + str(k)] = ix.get(str(lis[t]) + str(k),0) + 1
            col_ix[i+t*g] = ix[str(lis[t]) + str(k)]
        i += 1
 
    row_ix = np.repeat(np.arange(0,n),g)
    data = np.ones(nz)
    if p == None:
        p = len(ix)
 
    ixx = np.where(col_ix < p)
    return csr.csr_matrix((data[ixx],(row_ix[ixx],col_ix[ixx])),shape=(n,p)),ix

# 加载数据

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer

# laod data with pandas
cols = ['user', 'item', 'rating', 'timestamp']
train = pd.read_csv('http://files.grouplens.org/datasets/movielens/ml-100k/ua.base', delimiter='\t', names=cols)
test = pd.read_csv('http://files.grouplens.org/datasets/movielens/ml-100k/ua.test', delimiter='\t', names=cols)

# vectorize data and convert them to csr matrix
x_train,ix = vectorize_dic({'users':train['user'].values,
                            'items':train['item'].values},n=len(train.index),g=2)
 
 
x_test,ix = vectorize_dic({'users':test['user'].values,
                           'items':test['item'].values},ix,x_train.shape[1],n=len(test.index),g=2)
 
 
y_train = train['rating'].values
y_test = test['rating'].values
 
x_train = x_train.todense()
x_test = x_test.todense()


# 定义FM模型

In [3]:
import tensorflow as tf
n, p = x_train.shape
# # of factor
k = 10
X = tf.placeholder('float',[None,p])
y = tf.placeholder('float',[None,1])

# bias and weigths
w0 = tf.Variable(tf.zeros([1]))
w = tf.Variable(tf.zeros([p]))

V = tf.Variable(tf.random_normal([k,p],mean = 0,stddev = 0.01))

# y的估计量
y_hat = tf.Variable(tf.zeros([n,10]))

# 定义y的计算公式

In [4]:
from IPython.display import display, Math, Latex
display(Math(r'\hat{y}(\mathbf{x}) = w_0 + \sum_{j=1}^{p}w_jx_j + \frac{1}{2} \sum_{f=1}^{k} ((\sum_{j=1}^{p}v_{j,f}x_j)^2-\sum_{j=1}^{p}v_{j,f}^2 x_j^2)'))

<IPython.core.display.Math object>

In [5]:
 # 线性部分
linear_terms = tf.add(w0,tf.reduce_sum(tf.multiply(w,X),1,keep_dims=True))
# 组合特征部分
pair_interactions = 0.5 * tf.reduce_sum(
         tf.subtract(
        tf.pow(tf.matmul(X,tf.transpose(V)),2),
        tf.matmul(tf.pow(X,2),tf.transpose(tf.pow(V,2))))
      ,axis = 1 , keep_dims=True)

y_hat = tf.add(linear_terms,pair_interactions)                                

W0318 14:39:09.256244  7336 deprecation.py:506] From <ipython-input-5-23a1bb815007>:2: calling reduce_sum_v1 (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


# 定义损失函数
损失函数基本是一致的，但是$\hat y$在各个模型中的定义不同

In [6]:
display(Math(r'L = \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 + \lambda_w ||W||^2 + \lambda_v ||V||^2'))

<IPython.core.display.Math object>

In [7]:
# L2 regularized sum of squares loss function over W and V
lambda_w = tf.constant(0.001, name='lambda_w')
lambda_v = tf.constant(0.001, name='lambda_v')

l2_norm = (tf.reduce_sum(
            tf.add(
                tf.multiply(lambda_w, tf.pow(w , 2)),
                tf.multiply(lambda_v, tf.pow(w, 2)))))

error = tf.reduce_mean(tf.square(tf.subtract(y, y_hat)))
loss = tf.add(error, l2_norm)

# 优化

In [8]:
display(Math(r'\Theta_{i+1} = \Theta_{i} - \eta \frac{\delta L}{\delta \Theta}'))

<IPython.core.display.Math object>

In [9]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)

# 数据批处理

In [10]:
def batcher(X_, y_=None, batch_size=-1):
    n_samples = X_.shape[0]

    if batch_size == -1:
        batch_size = n_samples
    if batch_size < 1:
       raise ValueError('Parameter batch_size={} is unsupported'.format(batch_size))

    for i in range(0, n_samples, batch_size):
        upper_bound = min(i + batch_size, n_samples)
        ret_x = X_[i:upper_bound]
        ret_y = None
        if y_ is not None:
            ret_y = y_[i:i + batch_size]
            yield (ret_x, ret_y)

# 开启tensorflow并训练模型

In [13]:
epochs = 10
batch_size = 1000
 
# Launch the graph
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
 
    for epoch in tqdm(range(epochs), unit='epoch'):
        perm = np.random.permutation(x_train.shape[0])
        # iterate over batches
        for bX, bY in batcher(x_train[perm], y_train[perm], batch_size):
            _,t = sess.run([optimizer,loss], feed_dict={X: bX.reshape(-1, p), y: bY.reshape(-1, 1)})
            print(t)
 
 
    errors = []
    for bX, bY in batcher(x_test, y_test):
        errors.append(sess.run(error, feed_dict={X: bX.reshape(-1, p), y: bY.reshape(-1, 1)}))
        print(errors)
    RMSE = np.sqrt(np.array(errors).mean())
    print (RMSE)


13.666977
13.459277
11.895468
12.229354
11.978907
11.697511
10.898112
10.752986
10.290718
9.813786
9.807136
9.5477705
9.0315695
8.111083
8.3757
7.897618
7.987485
7.2583404
7.2744007
6.8828506
6.7953267
6.2840652
6.292376
6.002656
5.5803113
5.7798123
5.6434474
5.336013
5.000166
4.937323
4.778964
4.803615
4.535076
4.5546055
4.4564857
4.26334
4.0320787
4.000592
3.7109952
3.6614008
3.6036034
3.701316
3.7156446
3.391756
3.3318903
2.9437358
3.1904266
3.0303473
3.0058758
2.781895
2.7252507
2.7902694
2.8194306
2.6593657
2.5745451
2.5639713
2.5399594
2.496343
2.4049273
2.3603868
2.2856984
2.2806635
2.183684
2.0733094
2.128459
2.1011772
2.0609765
1.9393634
1.9505033
1.9703048
1.8953605
1.8605076
1.8673614
1.7454628
1.8196458
1.7379729
1.7835699
1.7304085
1.7636402
1.7235775
1.6770111
1.6989435
1.7189751
1.6492547
1.7453214
1.6447566
1.6604309
1.6269314
1.6431615
1.6631123
1.5384884
1.6034584
1.4558429
1.5337511
1.4424423
1.5770731
1.5240046
1.4888039
1.4044237
1.40075
1.4411513
1.524299
1.465733

1.2281094
1.2608694
1.254743
1.2039205
1.2427069
1.2659601
1.2707117
1.2360288
1.2394352
1.2804197
1.3515017
1.304545
1.2484348
1.4070529
1.2600833
1.2306561
1.2972014
1.2733343
1.2691842
1.3705152
1.1961622
1.2656883
1.2471582
1.3337793
1.2540843
1.3906453
1.2499503
1.3154749
1.2571304
1.3174648
1.3138297
1.2979499
1.1781074
1.2801539
1.3292255
1.31469
1.2696134
1.159064
1.1563606
1.2419709
1.2854751
1.2241461
1.3326541
1.1675496
1.1252637
1.1865844
1.2870222
1.2709103
1.3779898
1.1960359
1.2970524
1.2550465
1.3419309
1.3319819
1.2115636
1.3045535
1.3031638
1.3097249
1.2326785
1.309197
1.2528201
1.2576752
1.294744
1.1927041
1.3005142
1.251434
1.3150468
1.3097295
1.2267739
1.2047822
1.2299514
1.2610465
1.276422
1.2183188
1.1934199
1.2522523
1.1634479
1.3176885
1.1678786
1.2911397

[1.266807]
1.1255252
